# Creando el modelo de regresión

En este apartado cargaremos los datos históricos y crearemos un modelo de regresión para hacer los cálculos de predicción. Para este ejemplo, utilizaremos el modelo SVR (`Support Vector Regression` del módulo de [scikit-learn](https://scikit-learn.org/stable/index.html) de [Python](https://www.python.org/): [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)), pero se podrían utilizar otros modelos de regresión para comparar los resultados.

Los pasos que se seguirán en este *notebook* son los siguientes:

 1. [Carga de datos históricos](#Carga-de-datos-históricos)
 2. [Preparación de los datos](#Preparación-de-los-datos)
 3. [Normalización](#Normalización)
 4. [Añadiendo información a los datos de entrada](#Añadiendo-información-a-los-datos-de-entrada)
 5. [Training/Test/Validation Set](#Training/Test/Validation-Set)
 6. [Creando el modelo](#Creando-el-modelo)
 7. [Visualizando los resultados](#Visualizando-los-resultados)

## Carga de datos históricos

Vamos a empezar cargando los datos:

In [23]:
import pandas as pd

df = pd.read_csv('../Datos/TXT_Simulación_datos_2019-01-01_2019-12-31.txt',
                 parse_dates=['ticketDate'])

df.rename(columns={"ticketDate": "Fecha", "amount": "Importe"}, inplace=True)

df = df[df['Fecha'] >= '2018-01-01']

Nuestros `dataframe` de momento sólo contiene 2 columnas:

 - `Fecha`: día y hora de la emisión del ticket (formato `%Y-%m-%d %H:%M:%S`)
 - `Importe`: importe del ticket en €

In [24]:
df.tail()

,Fecha,Importe
131358,2019-12-31 23:07:00,702.53
131359,2019-12-31 23:16:00,730.08
131360,2019-12-31 23:19:00,830.46
131361,2019-12-31 23:28:00,781.43
131362,2019-12-31 23:50:00,652.99


y el tipo de dato que contiene cada columna:
 - `Fecha`: tipo de dato `datetime`
 - `Importe`: tipo de dato `float`

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131363 entries, 0 to 131362
Data columns (total 2 columns):
Fecha      131363 non-null datetime64[ns]
Importe    131363 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.0 MB


## Preparación de los datos

Como se ha indicado al inicio, el objetivo es crear un modelo de **regresión**, por lo que tenemos que preparar los datos de forma que tengamos unos datos de entrada $X$ y otros de salida $Y$.

Para ello, vamos a intentar transformar nuestro `dataframe` para que tenga un formato similar a la siguiente tabla (en donde las ventas se van acumulando cada 15 minutos):

| Dia | 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 | 22:00 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2019-09-26 | 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 | 5 000 150 |
| 2019-09-27 | 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 | 5 500 150 |
| 2019-09-28 | 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 | 6 000 150 |

De esta manera, tendríamos los datos de entrada $X$ (en el que se introducirán más variables):

| 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 |
| --- | --- | --- | --- | --- | --- |
| 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 |
| 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 |
| 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 |

Y los datos de salida $Y$:

| 22:00 |
| --- |
| 5 000 150 |
| 5 500 150 |
| 6 000 150 |

Este es un ejemplo ilustrativo, ya que se contabilizan compras fueras del rango usual laboral (compras online u horarios especiales).

Para preparar esa tabla, vamos a empezar agregando las ventas en intervalos de 15 minutos:

In [26]:
df.index = df.pop('Fecha')
df = df.resample('15T').sum()

df.tail()

,Importe
Fecha,
2019-12-31 22:45:00,3600.74
2019-12-31 23:00:00,702.53
2019-12-31 23:15:00,2341.97
2019-12-31 23:30:00,0.00
2019-12-31 23:45:00,652.99


Continuaremos desglosando la `Fecha` en 2 nuevas columnas:
 - `Dia`
 - `Hora`

In [27]:
df['Dia'] = df.index.map(lambda x: x.strftime('%Y-%m-%d'))
df['Hora'] = df.index.map(lambda x: x.strftime('%H:%M'))

df.tail()

,Importe,Dia,Hora
Fecha,,,
2019-12-31 22:45:00,3600.74,2019-12-31,22:45
2019-12-31 23:00:00,702.53,2019-12-31,23:00
2019-12-31 23:15:00,2341.97,2019-12-31,23:15
2019-12-31 23:30:00,0.00,2019-12-31,23:30
2019-12-31 23:45:00,652.99,2019-12-31,23:45


Ahora ya estamos listos para formatear nuestro `dataframe`, para ello utilizaremos la función [crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) del módulo de [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html):

In [28]:
df = pd.crosstab(index=df['Dia'],
                 columns=[df['Hora']],
                 values=df.Importe,
                 aggfunc=sum).fillna(0).reset_index()

df.set_index('Dia', inplace=True)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,3676.90,3024.49,4365.77,3683.28,4787.70,3357.76,1571.59,5170.48,1022.22,2347.76
2019-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,3074.57,1717.63,3215.97,2030.05,3392.85,2952.00,3647.96,3167.86,661.42,2860.67
2019-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.76,0.0,...,57.23,63.92,1.74,23.91,33.67,0.00,14.31,0.00,0.14,27.66
2019-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,3180.07,1698.02,2340.16,1816.42,1857.81,774.11,1139.71,444.54,367.55,796.98
2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,4171.09,1641.59,1552.60,2941.83,1503.32,3600.74,702.53,2341.97,0.00,652.99


Para finalizar con esta parte, vamos hacer la suma acumulada de cada fila:

In [29]:
df = df.cumsum(axis=1)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,229164.55,232189.04,236554.81,240238.09,245025.79,248383.55,249955.14,255125.62,256147.84,258495.60
2019-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,229777.12,231494.75,234710.72,236740.77,240133.62,243085.62,246733.58,249901.44,250562.86,253423.53
2019-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.76,24.76,...,3709.73,3773.65,3775.39,3799.30,3832.97,3832.97,3847.28,3847.28,3847.42,3875.08
2019-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,141881.30,143579.32,145919.48,147735.90,149593.71,150367.82,151507.53,151952.07,152319.62,153116.60
2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,253421.27,255062.86,256615.46,259557.29,261060.61,264661.35,265363.88,267705.85,267705.85,268358.84


## Normalización

Para la normalización de los datos vamos a dividir todos los campos por el máximo valor de la tabla. De esa manera, todos nuestros valores oscilarán entre 0 y 1.

In [30]:
max_value = df['23:45'].max()
df /= max_value
df.reset_index(inplace=True)

df.tail()

Hora,Dia,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
360,2019-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.806802,0.817451,0.832821,0.845788,0.862644,0.874465,0.879998,0.898202,0.901801,0.910066
361,2019-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.808959,0.815006,0.826328,0.833475,0.845420,0.855813,0.868656,0.879809,0.882138,0.892209
362,2019-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000087,...,0.013061,0.013286,0.013292,0.013376,0.013494,0.013494,0.013545,0.013545,0.013545,0.013643
363,2019-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.499511,0.505489,0.513728,0.520123,0.526663,0.529389,0.533401,0.534966,0.536260,0.539066
364,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.892201,0.897981,0.903447,0.913804,0.919097,0.931773,0.934247,0.942492,0.942492,0.944791


## Añadiendo información a los datos de entrada

Una vez que hemos preparado los datos para poder aplicar un modelo de regresión, vamos a añadir información adicional a los datos para intentar crear un modelo más afinado. En este caso vamos a añadir información de calendario:

 - Día de la semana
 - Días festivos
 
Esas nuevas variables van a ser variables **categóricas** por lo que crearemos [variables *dummy*](https://medium.com/hugo-ferreiras-blog/dealing-with-categorical-features-in-machine-learning-1bb70f07262d) para introducirlo en el modelo.

### Días de la semana

In [31]:
df['Dia'] = pd.to_datetime(df['Dia'])
weekdays = [
    [0, 'Lunes'],
    [1, 'Martes'],
    [2, 'Miercoles'],
    [3, 'Jueves'],
    [4, 'Viernes'],
    [5, 'Sabado'],
    [6, 'Domingo']
]
for weekday, weekday_name in weekdays:
    df[weekday_name] = df['Dia'].map(lambda x: x.weekday() == weekday)
    
df[['Dia', 'Lunes', 'Martes', 'Miercoles',
    'Jueves', 'Viernes', 'Sabado', 'Domingo']].tail()

Hora,Dia,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado,Domingo
360,2019-12-27,False,False,False,False,True,False,False
361,2019-12-28,False,False,False,False,False,True,False
362,2019-12-29,False,False,False,False,False,False,True
363,2019-12-30,True,False,False,False,False,False,False
364,2019-12-31,False,True,False,False,False,False,False


### Días Festivos

In [32]:
calendario = ['2019-01-01','2019-01-01','2019-01-06','2019-03-19',
              '2019-04-28','2019-05-15','2019-07-25','2019-08-15',
              '2019-10-12','2019-11-01','2019-12-06','2019-12-08','2019-12-25','2019-12-26']

df['Festivo'] = df['Dia'].isin(calendario)

## Training/Test/Validation Set

Por último, antes de crear el modelo vamos a dividir los datos en 3 bloques:

 - **Training Set**: 80% de los datos
 - **Test Set**: 10% de los datos
 - **Validation Set**: 10% de los datos

In [33]:
def training_test_set():
    # Training Set (80%)
    train_data = df.sample(frac=0.8, random_state=0)
    test_validation_data = df.drop(train_data.index)

    # Test/Validatin Set (10%/10%)
    test_data = test_validation_data.sample(frac=0.5, random_state=0)
    validation_data = test_validation_data.drop(test_data.index)

    # Definir la variable 'Y'
    train_y = train_data.pop('23:45')
    test_y = test_data.pop('23:45')
    validation_y = validation_data.pop('23:45')

    # Eliminamos la columna 'Dia'
    train_data.drop(['Dia'], axis=1, inplace=True)
    test_data.drop(['Dia'], axis=1, inplace=True)
    validation_data.drop(['Dia'], axis=1, inplace=True)
    
    return [train_data, test_data, validation_data, train_y, test_y, validation_y]

## Creando el modelo

Como se ha indicado en la introducción, en este ejemplo se va a utilizar el modelo [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) para la regresión y se va a elegir el `kernel` `rbf`.

No es objeto de este *notebook* explicar los detalles de este modelo, pero el lector que esté interesado en profundizar en este modelo puede ver con más detalle los algoritmos que se utilizan para hacer los cálculos en la documentación oficial de [scikit-learn](https://scikit-learn.org/stable/modules/svm.html#svm-regression).

En `SVR` hay básicamente 2 parámetros que se utilizan para ajustar el modelo:

 - `C`
 - `Epsilon`
 
a los que vamos a darle diferentes valores.

In [34]:
from sklearn.svm import SVR

def crear_modelo():
    error = -1

    for C in [0.1, 1, 100, 1000]:
        for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]:
            # Creamos el modelo con los parámetros seleccionados
            svr_rbf = SVR(kernel='rbf', C=C, gamma='auto', epsilon=epsilon)

            # Ajustamos el modelo a nuestros datos
            model = svr_rbf.fit(train_data, train_y)

            # Medir la calidad del modelo con el Test Set
            error_now = (model.predict(test_data) - test_y).std()

            # Guardar los parámetros si se ha mejorado el error
            if (error_now < error) or (error == -1):
                error = error_now
                C_good = C
                epsilon_good = epsilon
                
    return SVR(kernel='rbf', C=C_good, gamma='auto', epsilon=epsilon_good)

Como queremos crear diferentes modelos para cada hora, a la hora de crear el modelo tendremos que modificar las variables de entrada $X$, eliminando las columnas que no se van a tener en cuenta para el modelo. Para ese fin vamos a definir la siguiente función:

In [35]:
def eliminar_columnas(hour_now):
    """Eliminar las columnas que no se van a utilizar para el cálculo de las previsiones"""

    cols_drop = df.columns[(df.columns > hour_now) & (df.columns < '23:45')]
    for col in cols_drop:
        df.pop(col)

También vamos a necesitar otra función para guardar los modelos utilizando el módulo `joblib`

In [36]:
import os
import joblib


def guardar_modelo_svr(name):
    """Guardar los resultados del modelo"""

    folder = 'modelos/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    filename = 'all_data_model_' + name + '.sav'
    joblib.dump(model, folder + filename)

In [37]:
df_orig = df.copy()

Pongamos en marcha los cálculos...

In [38]:
for hour in range(10, 22):
    print('\nCreando modelo para las ' + str(hour) + '...')
    df = df_orig.copy()
    eliminar_columnas(str(hour) + ':00')
    train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()
    model = crear_modelo()
    model.fit(train_data, train_y)
    guardar_modelo_svr(str(hour) + '00')
    print('Modelo guardado!')
    print('Error de validación: ' + str((model.predict(validation_data) - validation_y).std()))
    
# Guardamos también el valor máximo 'max_value'
file = open('modelos/max_value.txt', 'w')
file.write(str(round(max_value, 2)))
file.close()


Creando modelo para las 10...
Modelo guardado!
Error de validación: 0.06616915075712082

Creando modelo para las 11...
Modelo guardado!
Error de validación: 0.03909782437499523

Creando modelo para las 12...
Modelo guardado!
Error de validación: 0.03429855184695769

Creando modelo para las 13...
Modelo guardado!
Error de validación: 0.031630889798067996

Creando modelo para las 14...
Modelo guardado!
Error de validación: 0.030395703962559147

Creando modelo para las 15...
Modelo guardado!
Error de validación: 0.026964580753134102

Creando modelo para las 16...
Modelo guardado!
Error de validación: 0.024550914547101538

Creando modelo para las 17...
Modelo guardado!
Error de validación: 0.019995470750504563

Creando modelo para las 18...
Modelo guardado!
Error de validación: 0.020646415421457624

Creando modelo para las 19...
Modelo guardado!
Error de validación: 0.015216776958637579

Creando modelo para las 20...
Modelo guardado!
Error de validación: 0.014425129567750593

Creando mode

## Visualizando los resultados

Cargamos las librerias de [Plotly](https://plot.ly/python/)

In [39]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

Seleccionamos los días que queremos mostrar

In [40]:
dias = df_orig.loc[df_orig.index.isin(validation_y.index), 'Dia']

Cargamos el modelo que queremos analizar:

In [41]:
hour = 10

# Cargamos el modelo
loaded_model = joblib.load('modelos/all_data_model_{}00.sav'.format(str(hour)))

# Creamos los datos de validación
df = df_orig.copy()
eliminar_columnas(str(hour) + ':00')
train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()

Visualizamos

In [42]:
iplot({
    'data': [go.Scatter(
                x=dias,
                y=validation_y * max_value,
                name='Real'
            ),go.Scatter(
                x=dias,
                y=loaded_model.predict(validation_data) * max_value,
                name='Predicción'
    )],
    'layout': go.Layout(
                yaxis={
                    'title': 'Ventas €'
                }
    )
})